#### add min_loss and early_stop

In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
train_data[:20]

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


In [4]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9


In [5]:
print(train_data.shape)
print(test_data.shape)

(4320, 27)
(4320, 11)


In [6]:
data=train_data.iloc[:,3:]
data=data.replace('NR',0)
data.shape

(4320, 24)

In [7]:
raw_data=data.values

20 天 * 24小时 * 18 feature => 18 , 20*24

In [8]:
year_data={}
for m in range(12):
    month_data=np.empty((18,480))
    for d in range(20):
        month_data[:,24*d:24*(d+1)]=raw_data[18*(m*20+d):18*(m*20+d+1),:]
    year_data[m]=month_data
print(year_data[0].shape)

(18, 480)


任务是根据9小时预测10小时,一个月有 20 天 * 24 小时 = 480 小时,按9小时分组,为 480-9=471组

In [9]:
x=np.empty((12*471,9*18))
y=np.empty((12*471,1))

In [10]:
for m in range(12):
    for d in range(471):      
#         print(d,year_data[m][:,9*d:9*(d+1)].shape)
        x[(m*471+d):(m*471+d+1),:]=year_data[m][:,d:9+d].reshape(1,-1)
        y[(m*471+d):(m*471+d+1),:]=year_data[m][9,9+d]

In [82]:
def linear_reg(x,w,b):
    return torch.mm(x,w)+b

In [83]:
def squared_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.size()))**2

In [84]:
def sgd(params,lr,batch_size):
    for p in params :
        p.data -= lr*p.grad/batch_size

In [85]:
def train_hand(params,num_epochs,x_train,y_train,x_test,y_test,lr,batch_size):
    train_l,test_l=[],[]
    dataset=torch.utils.data.TensorDataset(x_train,y_train)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for e in range(num_epochs):
        for x,y in data_iter:
            l=squared_loss(linear_reg(x,params[0],params[1]),y).mean()
            if params[0].grad is not None:
                for p in params:
                    p.grad.data.zero_()
#                 w.grad.data.zero_()
#                 b.grad.data.zero_()
            l.backward()
            sgd(params,lr,batch_size)
        if (e+1) % 200 ==0:
            train_l.append(squared_loss(linear_reg(x_train,params[0],params[1]),y_train).mean().item())
            if x_test is not None:
                with torch.no_grad():
                    test_l.append(squared_loss(linear_reg(x_test,params[0],params[1]),y_test).mean().item())
    return train_l,test_l
                

In [86]:
def get_kfold_data(k,j,x_data,y_data):
    fold_size=x.shape[0]//k
#     idx=list(range(x.shape[0]))
#     np.random.shuffle(idx)
    x_train,y_train=None,None
    for i in range(k):
        x_part=x_data[i*fold_size:(i+1)*fold_size,:]
        y_part=y_data[i*fold_size:(i+1)*fold_size,:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val
            
        

In [87]:
def train_kfold(k,num_epochs,features,labels,lr,batch_size):
    train_l_sum,test_l_sum=[],[]
    for j in range(k):
        w=torch.Tensor(np.random.normal(0,0.01,(features.shape[1],1)))
        b=torch.Tensor([1.])
        for p in [w,b]:
            p.requires_grad_(requires_grad=True)
        params=[w,b]
        data=get_kfold_data(k,j,features,labels)
        train_l,test_l=train_hand(params,num_epochs,*data,lr,batch_size)
        train_l_sum.append(np.mean(train_l))
        test_l_sum.append(np.mean(test_l))
        print('%d fold:No. %d,train loss : %f, test loss : %f' % 
              (k,j,train_l_sum[-1],test_l_sum[-1]))
    print('%d fold train loss avg : %f,test loss avg : %f ' % (k,np.mean(train_l_sum)
                                                              ,np.mean(test_l_sum)))
    

In [88]:
w=torch.Tensor(np.random.normal(0,0.01,(features.shape[1],1)))
b=torch.Tensor([1.])
for p in [w,b]:
    p.requires_grad_(requires_grad=True)
features=torch.Tensor(x1.values)
labels=torch.Tensor(y)
k=5
num_epochs=5000
lr=0.03
batch_size=128
train_kfold(k,num_epochs,features,labels,lr,batch_size)

5 fold:No. 0,train loss : 32.944531, test loss : 42.002495
5 fold:No. 1,train loss : 32.765359, test loss : 43.024784
5 fold:No. 2,train loss : 32.323253, test loss : 45.014255
5 fold:No. 3,train loss : 37.132110, test loss : 22.804949
5 fold:No. 4,train loss : 34.597182, test loss : 34.595697
5 fold train loss avg : 33.952487,test loss avg : 37.488436 


In [38]:
np.sqrt(33.17)

5.759340239992772

add active func

In [69]:
def relu(x):
    return torch.max(x,torch.Tensor([0.]))

add more layers

In [89]:
def linear_reg2(x,w1,b1,w2,b2):
    h1= relu(torch.mm(x,w1)+b1)
    return torch.mm(h1,w2)+b2

In [102]:
def train_hand2(params,num_epochs,x_train,y_train,x_test,y_test,lr,batch_size):
    train_l,test_l=[],[]
    dataset=torch.utils.data.TensorDataset(x_train,y_train)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    min_loss=1000.
    early_stop_cnt=0
    for e in range(num_epochs):
        for x,y in data_iter:
            l=squared_loss(linear_reg2(x,params[0],params[1],params[2],params[3]),y).mean()
            if params[0].grad is not None:
                for p in params:
                    p.grad.data.zero_()
            l.backward()
            sgd(params,lr,batch_size)
        if (e+1) % 200 ==0:
            train_l.append(squared_loss(linear_reg2(x_train,params[0],params[1],params[2],params[3]),y_train).mean().item())
            if x_test is not None:
                with torch.no_grad():
                    test_l.append(squared_loss(linear_reg2(x_test,params[0],params[1],params[2],params[3]),y_test).mean().item())
                    
    return train_l,test_l
                

In [105]:
def train_kfold2(k,num_epochs,features,labels,lr,batch_size):
    train_l_sum,test_l_sum=[],[]
    for j in range(k):
        w1=torch.Tensor(np.random.normal(0,0.01,(features.shape[1],64)))
        w2=torch.Tensor(np.random.normal(0,0.01,(64,1)))
        b1=torch.Tensor(np.ones(64))
        b2=torch.Tensor([1.])
        for p in [w1,b1,w2,b2]:
            p.requires_grad_(requires_grad=True)
        params=[w1,b1,w2,b2]
        data=get_kfold_data(k,j,features,labels)
        train_l,test_l=train_hand2(params,num_epochs,*data,lr,batch_size)
        train_l_sum.append(np.mean(train_l))
        test_l_sum.append(np.mean(test_l))
        print('%d fold:No. %d,train loss : %f, test loss : %f' % 
              (k,j,train_l_sum[-1],test_l_sum[-1]))
    print('%d fold train loss avg : %f,test loss avg : %f ' % (k,np.mean(train_l_sum)
                                                              ,np.mean(test_l_sum)))
    

In [106]:
w1=torch.Tensor(np.random.normal(0,0.01,(features.shape[1],64)))
w2=torch.Tensor(np.random.normal(0,0.01,(64,1)))
b1=torch.Tensor(np.ones(64))
b2=torch.Tensor([1.])
for p in [w1,b1,w2,b2]:
    p.requires_grad_(requires_grad=True)
features=torch.Tensor(x1.values)
labels=torch.Tensor(y)
k=5
num_epochs=1000
lr=0.01
batch_size=128
train_kfold2(k,num_epochs,features,labels,lr,batch_size)

5 fold:No. 0,train loss : 31.898910, test loss : 40.648410
5 fold:No. 1,train loss : 32.294398, test loss : 42.337601
5 fold:No. 2,train loss : 30.990164, test loss : 45.626721
5 fold:No. 3,train loss : 35.700326, test loss : 23.395190
5 fold:No. 4,train loss : 33.496547, test loss : 36.045113
5 fold train loss avg : 32.876069,test loss avg : 37.610607 
